# Setup

Add Xavier to the path so you can import local modules from remote jupyter server

In [7]:
import sys
sys.path.append('Xavier')
#Install modules and mount drive
import alpaca_trade_api as apca
import websocket, json, asyncio, datetime, time, options_data
import matplotlib.pyplot as plt
import mplfinance.original_flavor as mplf
import matplotlib.dates as mpdates
import pandas as pd
import imp, math
import polygon as p
import yahoo_fin.options as yfo
import tda
import accounts, config, tickers
from tda.orders import options
import accounts, tickers, config, watchlist
from alpaca_trade_api import TimeFrame
from tda import orders
from options_data import get_options_high_low
import numpy as np
client = tda.auth.client_from_token_file(f'tokens/kiran.json',config.TD_CLIENT_ID)

Create the different types of accounts and define the 'get_td_option_name' function

In [3]:
#Create Trade Account Class
class trade_account:
    name = ''
    account_num = 0
    account_value = 0
    percent = 0
    allocation = 0
    contract_counts = {}
    def __init__(self,name,account_num,percent=0.9):
      self.name = name
      self.account_num = account_num
      self.percent = percent
      client = tda.auth.client_from_token_file(f'tokens/{name}.json',config.TD_CLIENT_ID)
      self.account_value = client.get_account(self.account_num).json()['securitiesAccount']['currentBalances']['cashAvailableForTrading']
      self.contract_counts = {}
    def add_to_list(self):
      accounts.accounts[self.name] = self.account_num
    def get_account_value(self):
      client = tda.auth.client_from_token_file(f'tokens/{self.name}.json',config.TD_CLIENT_ID)
      self.account_value = client.get_account(self.account_num).json()['securitiesAccount']['currentBalances']['cashAvailableForTrading']
      return self.account_value
    def get_allocation(self,numstocks):
      self.allocation = math.floor(math.floor(self.account_value*self.percent)/numstocks)
    def get_raw_balances(self):
      client = tda.auth.client_from_token_file(f'tokens/{self.name}.json',config.TD_CLIENT_ID)
      return client.get_account(self.account_num).json()
    def buy_option(self,symbol:str,price:float,verbose:bool):
        numcontracts = math.floor(self.allocation/(price*100))
        try:
          print(f'Numcontracts {numcontracts}')
          self.contract_counts[symbol] = numcontracts
          if numcontracts!=0:
            orderspec = options.option_buy_to_open_limit(symbol,numcontracts,price=price)
            client = tda.auth.client_from_token_file(f'tokens/{self.name}.json',config.TD_CLIENT_ID)
            message = client.create_saved_order(self.account_num,orderspec)
            #message = client.place_order(self.account_num,orderspec).content
            if verbose: 
              print(message,f'Bought {symbol} for price {price} at {datetime.datetime.now().strftime("%m%d%Y")}')
          return price, numcontracts
        except:
          print(f'{self.name} too small allocation to buy option {symbol} for {numcontracts*price} with allocation {self.allocation}')
    def sell_option(self,symbol:str,verbose:bool,price:float,boughtprice:float):
        numcontracts = self.contract_counts[symbol]
        runners = 0
        if numcontracts>20 and price>boughtprice:
            runners = 2
        print(symbol,numcontracts,f'runners: {runners}')
        orderspec = options.option_sell_to_close_limit(symbol,numcontracts-runners,price=price)
        client = tda.auth.client_from_token_file(f'tokens/{self.name}.json',config.TD_CLIENT_ID)
        message = client.create_saved_order(self.account_num,orderspec)
        #message = client.place_order(self.account_num,orderspec).json()
        if verbose:
          print(message)
    def cancel_orders(self):
        client = tda.auth.client_from_token_file(f'tokens/{self.name}.json',config.TD_CLIENT_ID)
        orderlist = [x['orderId'] for x in client.get_orders_by_query(statuses = [client.Order.Status('WORKING')]).json()]
        for x in orderlist:
            client.cancel_order(x,self.account_num)

class paper_trade_account:
    name = ''
    account_value = 0
    percent = 0
    allocation = 0
    contract_counts = {}
    def __init__(self,name,account_value,percent=0.9):
      self.name = name
      self.percent = percent
      self.account_value = account_value
      self.contract_counts = {}
    def get_balance(self):
      return self.account_value
    def get_allocation(self,numstocks):
      self.allocation = math.floor(math.floor(self.account_value*self.percent)/numstocks)
    def get_raw_balances(self):
      return self.account_value
    def buy_option(self,symbol:str,price:float,verbose:bool):
        numcontracts = math.floor(self.allocation/(price*100))
        try:
          print(f'Numcontracts {numcontracts}')
          self.contract_counts[symbol] = numcontracts
          if numcontracts!=0:
            if verbose: 
              print(f'Bought {symbol} for price {price} at {datetime.datetime.now().strftime("%m%d%Y")}')
          self.account_value -= price*numcontracts*100
          return price, numcontracts
        except:
          print(f'{self.name} too small allocation to buy option {symbol} for {numcontracts*price} with allocation {self.allocation}')
    def sell_option(self,symbol:str,verbose:bool,price:float,boughtprice:float):
        numcontracts = self.contract_counts[symbol]
        runners = 0
        if numcontracts>20 and price>boughtprice:
            runners = 0
        print(symbol,numcontracts,f'runners: {runners}')
        self.contract_counts[symbol] = runners
        self.account_value+= price*numcontracts*100
    def get_account_value(self):
        value = float(self.account_value)
        for x,y in self.contract_counts.items():
            client = tda.auth.client_from_token_file(f'tokens/kiran.json',config.TD_CLIENT_ID)
            price = client.get_quote(x).json()[x]['mark']
            value+=y*price*100
        return value
    def cancel_orders(self):
        pass
def get_td_option_name(ticker:str,expiration:str,cp:str,strike:int):
    assert cp.upper() in ('C','P')
    return tda.orders.options.OptionSymbol(ticker,datetime.datetime.strptime(expiration,'%B %d, %Y').strftime('%m%d%y'),cp.upper(),str(strike)).build()

# Stock Class

In [4]:
#Create Stock Class
apcarest = apca.REST(config.APCA_KEY_ID,config.APCA_SECRET_KEY)
class stock:
    ticker = ""
    stream = ""
    key = "c7i7ajqad3if83qgf1l0"
    today = datetime.datetime.now().date()
    startdate = today-datetime.timedelta(days=7)
    startts = int(time.mktime(startdate.timetuple()))
    tstoday = int(time.mktime(today.timetuple()))
    candleone = {}
    candletwo = {}
    minutecount = 0
    toplimit, bottomlimit = 0,0
    optionName = ""
    td_option_name = ""
    callBoughtPrice,putBoughtPrice = 0,0
    stopLoss,takeProfit = 0,0
    soldCallPrice, soldPutPrice = 0,0
    putStrike, callStrike = 0,0
    TPpct, SLpct = 0,0
    expiration = 0
    ioc = False
    oib = False
    ttl = False
    tbl = False
    oacot = False
    oacub = False
    trend = None
    sldict = {}
    checklist = {
        "InOutCandles":ioc,
        "Opened in between":oib,
        "Touches Top Line":ttl,
        "Touches Bottom Line":tbl,
        "Open and close over top":oacot,
        "Open and close under bottom":oacub
    }
    candleplot = 0
    plotdata = pd.DataFrame()
    fig,ax = (0,0)
    dateformat = mpdates.DateFormatter("%d-%m-%Y")
    contract = {"b":"","t":"","s":""}
    candlesdf = pd.DataFrame(columns=  ["Symbol","Open","High","Low","Close","Volume","Start time","End time","Start datetime","End datetime"])
    def from_old(self,old):
        for x in ['toplimit','bottomlimit','optionName','td_option_name','TPpct','SLpct','callBoughtPrice','putBoughtPrice','soldCallPrice','soldPutPrice','callStrike','putStrike','ioc','oib','tbl','oacub','ttl','oacot']:
            setattr(self,x,getattr(old,x))
    def get_ioc(self,logs=[]):
        logs.append(datetime.datetime.now().timestamp(), f'{self.ticker} ioc : {self.ioc}')
        return self.ioc
    def get_oib(self,logs=[]):
        logs.append(datetime.datetime.now().timestamp(), f'{self.ticker} oib : {self.oib}')
        return self.oib
    def get_ttl(self,logs=[]):
        logs.append(datetime.datetime.now().timestamp(), f'{self.ticker} ttl : {self.ttl}')
        return self.ttl
    def get_tbl(self,logs=[]):
        logs.append(datetime.datetime.now().timestamp(), f'{self.ticker} tbl : {self.tbl}')
        return self.tbl
    def get_oacot(self,logs=[]):
        logs.append(datetime.datetime.now().timestamp(), f'{self.ticker} oacot : {self.oacot}')
        return self.oacot
    def get_oacub(self,logs=[]):
        logs.append(datetime.datetime.now().timestamp(), f'{self.ticker} oacub : {self.oacub}')
        return self.oacub
    def set_ioc(self,val,logs=[],verbose = False):
        if verbose and self.ioc!=val:
            print(datetime.datetime.now(), f'{self.ticker} set ioc : {val}')
        self.ioc = val
        logs.append(datetime.datetime.now().timestamp(), f'{self.ticker} set ioc : {val}')
    def set_oib(self,val,logs=[],verbose = False):
        if verbose and self.oib!=val:
            print(datetime.datetime.now(), f'{self.ticker} set oib : {val}')
        self.oib = val
        logs.append(datetime.datetime.now().timestamp(), f'{self.ticker} set oib : {val}')
    def set_ttl(self,val,logs=[],verbose = False):
        if verbose and self.ttl!=val:
            print(datetime.datetime.now(), f'{self.ticker} set ttl : {val}')
        self.ttl = val
        logs.append(datetime.datetime.now().timestamp(), f'{self.ticker} set ttl : {val}')
    def set_tbl(self,val,logs=[],verbose = False):
        if verbose and self.tbl!=val:
            print(datetime.datetime.now(), f'{self.ticker} set tbl : {val}')
        self.tbl = val
        logs.append(datetime.datetime.now().timestamp(), f'{self.ticker} set tbl : {val}')
    def set_oacot(self,val,logs=[],verbose = False):
        if verbose and self.oacot!=val:
            print(datetime.datetime.now(), f'{self.ticker} set oacot : {val}')
        self.oacot = val
        logs.append(datetime.datetime.now().timestamp(), f'{self.ticker} set oacot : {val}')
    def set_oacub(self,val,logs=[],verbose = False):
        if verbose and self.oacub!=val:
            print(datetime.datetime.now(), f'{self.ticker} set oacub : {val}')
        self.oacub = val
        logs.append(datetime.datetime.now().timestamp(), f'{self.ticker} set oacub : {val}')
    def set_SLpct(self,val,logs=[],verbose = False):
        if verbose and self.SLpct!=val:
            print(datetime.datetime.now(), f'{self.ticker} set SLpct : {val}')
        self.SLpct = 1.0-val
        logs.append(datetime.datetime.now().timestamp(), f'{self.ticker} set SLpct : {val}')
    def set_TPpct(self,val,logs=[],verbose = False):
        if verbose and self.TPpct!=val:
            print(datetime.datetime.now(), f'{self.ticker} set TPpct : {val}')
        self.TPpct = 1.0+val
        logs.append(datetime.datetime.now().timestamp(), f'{self.ticker} set TPpct : {val}')
    def __init__(self,ticker,allocation,TPpct=.3,SLpct=.2, sldict={5:.15,10:.1,20:.5}) -> None:
        self.ticker = ticker.upper()
        self.stream = "AM."+ticker.upper()
        self.TPpct = 1.0+TPpct
        self.SLpct = 1.0-SLpct
        self.allocation = allocation
        self.sldict = sldict
    def addCandle(self,candle):
        self.candlesdf = self.candlesdf.append(candle)
    def getPastTwoDaysCandles(self,sameday=False):
        if sameday:
            end = datetime.datetime.today().strftime(f'%Y-%m-%d')
        else:
            end = (datetime.datetime.today()-datetime.timedelta(1)).strftime(f'%Y-%m-%d')
        resp = apcarest.get_bars(self.ticker,TimeFrame.Day,start=(datetime.datetime.today()-datetime.timedelta(7)).strftime(f'%Y-%m-%d'),end=end).df
        self.candleone = resp.iloc[-2]
        self.candletwo = resp.iloc[-1]
        self.plotdata = pd.DataFrame(resp)
    def showplot(self):
        self.fig,self.ax = plt.subplots()
        self.ax.grid(True)
        plt.title(self.ticker)
        self.ax.xaxis.set_major_formatter(self.dateformat)
        plotdf = self.plotdata
        plotdf ['t'] = plotdf["t"].map(datetime.datetime.fromtimestamp)
        plotdf ['t'] = plotdf["t"].map(mpdates.date2num)
        mplf.candlestick_ohlc(self.ax,self.plotdata.values,width=0.2,colorup="green",colordown="red",alpha=0.8)
        self.fig.autofmt_xdate()
        self.fig.tight_layout()
    def checkPastTwoDays(self,updateself=True,trendlength=6,trend=.01,logs=[]):
        #assert self.candleone!={} and self.candletwo!={},"Run getPastTwoDaysCandles function first"
        candleone, candletwo = self.candleone,self.candletwo
        client = tda.auth.client_from_token_file(f'tokens/kiran.json',config.TD_CLIENT_ID)
        trend_points = pd.DataFrame(client.get_price_history_every_day(self.ticker).json()['candles'])['close'][-trendlength:]
        actual_trend,_ = np.polyfit(trend_points,[x for x in range(trendlength)],1)
        if actual_trend>trend:
            self.trend='up'
        if actual_trend<-trend:
            self.trend='down'
        #set limits if its an inside outside candle
        if (candletwo['low']>candleone['low'] and candletwo['high']<candleone['high']):
            templist = [candletwo['low'],candletwo['high']]
            templist.sort()
            self.bottomlimit,self.toplimit = templist
            logs.append(datetime.datetime.now().timestamp(),f"Limits for {self.ticker}: {self.toplimit} | {self.bottomlimit}")
            if updateself:
                self.set_ioc(True,verbose=True,logs=logs)
            return True
        else:
            return False
    def checkOpen(self, logs):
        assert self.toplimit!=self.bottomlimit 
        opents = int(datetime.datetime.combine(datetime.datetime.today().date(),datetime.time(hour = 9,minute = 30)).timestamp())
        with apca.REST(config.APCA_KEY_ID,config.APCA_SECRET_KEY) as apcarest:
            high,low = apcarest.get_bars(symbol = self.ticker, timeframe=apca.TimeFrame.Day).df[['high','low']].values[0]
            logs.append(datetime.datetime.now().timestamp(),f"{self.ticker} opened in between: {low<self.toplimit and high> self.bottomlimit}")
            return low<self.toplimit and high> self.bottomlimit
    def updateChecklist(self,ropen,rhigh,rlow,rclose,logs=[]):
        beginChklst = self.checklist.copy()
        if self.get_oib(logs=logs) == False:
            self.set_oib(rhigh<self.toplimit and rlow>self.bottomlimit,logs=logs,verbose=True)
        self.set_oacot(ropen>self.toplimit and rclose>self.toplimit,verbose=True,logs=logs)
        self.set_oacub(ropen<self.bottomlimit and rclose<self.bottomlimit,verbose=True,logs=logs)
        self.set_ttl(rhigh>self.toplimit,verbose=True,logs=logs)
        self.set_tbl(rlow<self.bottomlimit,verbose=True,logs=logs)
        self.checklist = {
            "InOutCandles":self.ioc,
            "Opened in between":self.oib,
            "Touches Top Line":self.ttl,
            "Touches Bottom Line":self.tbl,
            "Open and close over top":self.oacot,
            "Open and close under bottom":self.oacub
        }
        if beginChklst!=self.checklist:
            return True
        else:
            return False
    def updateChecklistBase(self,logs=[]):
        _,ropen,rhigh,rlow,rclose,volume,_,_ = apcarest.get_latest_bar(self.ticker)._raw.values()
        ts = datetime.datetime.now().timestamp()
        self.addCandle(pd.DataFrame([[self.ticker,ropen,rhigh,rlow,rclose,volume,ts,ts,datetime.datetime.fromtimestamp(ts/1000),datetime.datetime.fromtimestamp(ts/1000)]],columns = ["Symbol","Open","High","Low","Close","Volume","Start time","End time","Start datetime","Edn datetime"]))
        beginChklst = self.checklist.copy()
        if self.get_oib(logs=logs) == False:
            self.set_oib(rhigh<self.toplimit and rlow>self.bottomlimit,logs=logs,verbose=True)
        self.set_oacot(ropen>self.toplimit and rclose>self.toplimit,verbose=True,logs=logs)
        self.set_oacub(ropen<self.bottomlimit and rclose<self.bottomlimit,verbose=True,logs=logs)
        self.set_ttl(rhigh>self.toplimit,verbose=True,logs=logs)
        self.set_tbl(rlow<self.bottomlimit,verbose=True,logs=logs)
        self.checklist = {
            "InOutCandles":self.ioc,
            "Opened in between":self.oib,
            "Touches Top Line":self.ttl,
            "Touches Bottom Line":self.tbl,
            "Open and close over top":self.oacot,
            "Open and close under bottom":self.oacub
        }
        if beginChklst!=self.checklist:
            return True
        else:
            return False
    def reload_checklist(self):
        self.checklist = {
            "InOutCandles":self.ioc,
            "Opened in between":self.oib,
            "Touches Top Line":self.ttl,
            "Touches Bottom Line":self.tbl,
            "Open and close over top":self.oacot,
            "Open and close under bottom":self.oacub
        }
    def updateStopLoss(self,cp,logs,verbose:bool=True):
        if cp.lower() == "call":
            client = tda.auth.client_from_token_file(f'tokens/kiran.json',config.TD_CLIENT_ID)
            price = client.get_quote(self.td_option_name).json()[self.td_option_name]['mark']
            newsl = self.stopLoss
            for x,y in self.sldict.items():
                if self.minutecount>=x: newsl = self.callBoughtPrice*(1-y)
            if newsl>self.stopLoss and round(price/self.callBoughtPrice,2)/.05>0:
                logs.append(datetime.datetime.now().timestamp(),f'Change stoploss of {self.ticker} | {self.optionName} from {self.stopLoss} to {newsl}')
                if verbose:
                  print(f'Change stoploss of {self.ticker} | {self.optionName} from {self.stopLoss} to {newsl}')
                self.stopLoss = newsl
            return self.stopLoss
        elif cp.lower() == 'put':
            client = tda.auth.client_from_token_file(f'tokens/kiran.json',config.TD_CLIENT_ID)
            price = client.get_quote(self.td_option_name).json()[self.td_option_name]['mark']
            newsl = self.stopLoss
            for x,y in self.sldict.items():
                if self.minutecount>=x: newsl = self.callBoughtPrice*(1-y)
            if newsl>self.stopLoss and round(price/self.putBoughtPrice,2)/.05>0:
                logs.append(datetime.datetime.now().timestamp(),f'Change stoploss of {self.ticker} | {self.optionName} from {self.stopLoss} to {newsl}')
                if verbose:
                  print(f'Change stoploss of {self.ticker} | {self.optionName} from {self.stopLoss} to {newsl}')
                self.stopLoss = newsl
            return self.stopLoss
        else:
            return 'cp must be either "call" or "put"'
    def buy_call(self,accountslist:list,logs=[],verbose=False):
        #TODO
        client = tda.auth.client_from_token_file(f'tokens/kiran.json',config.TD_CLIENT_ID)
        isFriThur = datetime.datetime.today().weekday() == 4 or datetime.datetime.today().weekday() == 3
        expiration = 1 if isFriThur else 0
        calldict = pd.DataFrame(pd.DataFrame(client.get_option_chain(self.ticker,strike_count=4).json()).iloc[expiration]['callExpDateMap']).iloc[0].iloc[-1]
        self.expiration = expiration
        currentprice = apcarest.get_latest_quote(self.ticker)._raw['ap']
        client = tda.auth.client_from_token_file(f'tokens/kiran.json',config.TD_CLIENT_ID)
        td_option_name, strike, price, expdate = pd.Series(calldict)[['symbol','strikePrice','mark','expirationDate']].values
        option_name = self.ticker+datetime.datetime.fromtimestamp(expdate/1000).strftime('%y%m%d')+"C"+"%08.0f"%(strike*1000)
        self.td_option_name = td_option_name
        self.callStrike = strike
        self.optionName = option_name
        for x in accountslist:
            try:
                print(self.td_option_name,price)
                price,numcontracts = x.buy_option(self.td_option_name,price=price,verbose = True)
                higher = max(self.candleone[['open','close']])
                if self.candletwo.high<higher:
                    self.takeProfit = higher
                else:
                    self.takeProfit = self.candleone
                self.callBoughtPrice = price
                self.stopLoss = (self.candleone['high']+self.candleone['low'])/2.0
                #self.stopLoss = price*self.SLpct
                #self.takeProfit = price*self.TPpct
                if verbose:
                    print(option_name,f"price:{price}", f"strike: {strike}", datetime.datetime.now())
            except:
                continue
        if verbose:
            print(f'Stop loss: {self.stopLoss} | Take profit: {self.takeProfit}')
        logs.append(datetime.datetime.now().timestamp(),f'{self.ticker} price: {currentprice} bought call: {option_name} for strike of {strike} and price of {price}. Stop loss: {self.stopLoss} | Take profit: {self.takeProfit}')
        return option_name

    def buy_put(self,accountslist:list,logs=[],verbose=False):
        #TODO
        client = tda.auth.client_from_token_file(f'tokens/kiran.json',config.TD_CLIENT_ID)
        isFriThur = datetime.datetime.today().weekday() == 4 or datetime.datetime.today().weekday() == 3
        expiration = 1 if isFriThur else 0
        putdict = pd.DataFrame(pd.DataFrame(client.get_option_chain(self.ticker,strike_count=4).json()).iloc[expiration]['putExpDateMap']).iloc[0].iloc[0]
        self.expiration = expiration
        currentprice = apcarest.get_latest_quote(self.ticker)._raw['ap']
        client = tda.auth.client_from_token_file(f'tokens/kiran.json',config.TD_CLIENT_ID)
        td_option_name, strike, price, expdate = pd.Series(putdict)[['symbol','strikePrice','mark','expirationDate']].values
        option_name = self.ticker+datetime.datetime.fromtimestamp(expdate/1000).strftime('%y%m%d')+"P"+"%08.0f"%(strike*1000)
        self.td_option_name = td_option_name
        self.putStrike = strike
        self.optionName = option_name
        for x in accountslist:
            try:
              print(self.td_option_name,price)
              price,numcontracts = x.buy_option(self.td_option_name,price=price,verbose = True)
              lower = min(self.candleone.open,self.candleone.close)
              if self.candletwo.high>lower:
                   self.takeProfit = lower
              else:
                   self.takeProfit = self.candleone.high
              self.putBoughtPrice = price
              self.stopLoss = (self.candleone['high']+self.candleone['low'])/2.0
              #self.stopLoss = price*self.SLpct
              #self.takeProfit = price*self.TPpct
              if verbose:
                  print(option_name,f"price:{price}", f"strike: {strike}", datetime.datetime.now())
            except:
              continue
        if verbose:
            print(f'Stop loss: {self.stopLoss} | Take profit: {self.takeProfit}')
        logs.append(datetime.datetime.now().timestamp(),f'{self.ticker} price: {currentprice} bought put: {option_name} for strike of {strike} and price of {price}. Stop loss: {self.stopLoss} | Take profit: {self.takeProfit}')
        return option_name

    def check_sell_call(self,accountslist:list,logs=[],verbose=False):
        expiration = self.expiration
        client = tda.auth.client_from_token_file(f'tokens/kiran.json',config.TD_CLIENT_ID)
        price = client.get_quote(self.ticker).json()[self.ticker]['mark']
        option_price = client.get_quote(self.td_option_name).json()[self.td_option_name]['mark']
        if price >= self.takeProfit or price<= self.stopLoss:
          for x in accountslist:
              try:
                print(self.td_option_name,option_price)
                x.sell_option(self.td_option_name,verbose = True,price = option_price,boughtprice = self.callBoughtPrice)
                self.minutecount=0
                self.soldCallPrice = option_price
              except:
                continue
          profit = option_price-self.callBoughtPrice
          percent_profit = profit/self.callBoughtPrice
          if verbose: print(f'Sell {self.optionName} at {datetime.datetime.now()} for a profit of {profit}')
          logs.append(datetime.datetime.now().timestamp(),f'Sell {self.optionName} at {datetime.datetime.now()} for {price}. Profit: {profit} | {percent_profit}%')
          return option_price
        else: return False

    def check_sell_put(self,accountslist:list,logs=[],verbose=False):
        expiration = self.expiration
        client = tda.auth.client_from_token_file(f'tokens/kiran.json',config.TD_CLIENT_ID)
        price = client.get_quote(self.ticker).json()[self.ticker]['mark']
        option_price = client.get_quote(self.td_option_name).json()[self.td_option_name]['mark']
        if price <= self.takeProfit or price>= self.stopLoss:
          for x in accountslist:
              try:
                print(self.td_option_name,option_price)
                x.sell_option(self.td_option_name,verbose = True,price = option_price, boughtprice = self.putBoughtPrice)
                self.minutecount=0
                self.soldPutPrice = option_price
              except:
                continue
          profit = option_price-self.putBoughtPrice
          percent_profit = profit/self.putBoughtPrice
          if verbose: print(f'Sell {self.optionName} at {datetime.datetime.now()} for a profit of {profit}')
          logs.append(datetime.datetime.now().timestamp(),f'Sell {self.optionName} at {datetime.datetime.now()} for {price}. Profit: {profit} | {percent_profit}%')
          return price
        else: return False
    
class log:
    log = pd.DataFrame(columns=["Time","Message"])
    def __init__(self):
      self.log = pd.DataFrame(columns=["Time","Message"])
    def append(self,time,message):
      self.log.loc[len(self.log.index)] = [time,message]
    def datetime_format(self):
        datetimeCol = [datetime.datetime.fromtimestamp(x) for x in self.log.Time.values]
        return pd.DataFrame({'Time':datetimeCol,'Message':self.log.Message})


# Define functions for websocket

In [5]:
options_dict = {}

In [6]:
last_save = 0
def on_open(ws):
    print("opened")
    auth_data = {
        "action": "auth",
        "key": config.APCA_KEY_ID, 
        "secret": config.APCA_SECRET_KEY
    }

    ws.send(json.dumps(auth_data))

#This is the line that tells what tickers to listen to 
    listen_message = {"action": "subscribe", "bars":[x.ticker for x in tickDict.values()]}

    ws.send(json.dumps(listen_message))
    
def on_message(ws, message):
    start_time = time.time()
    print('received')
    global last_save
    decoder = json.decoder.JSONDecoder()
    jsonmessage = (decoder.decode(message))[0]
    try:
        tick = jsonmessage['S']
        open = jsonmessage['o']
        high = jsonmessage['h']
        low = jsonmessage['l']
        close = jsonmessage['c']
        volume = jsonmessage['v']
        print(tick)
        startts,endts = datetime.datetime.now().timestamp(), (datetime.datetime.now()-datetime.timedelta(minutes=1)).timestamp()
        candle = pd.DataFrame(data = [[tick,open,high,low,close,volume,startts,endts,datetime.datetime.fromtimestamp(startts/1000),datetime.datetime.fromtimestamp(endts/1000)]],columns= ["Symbol","Open","High","Low","Close","Volume","Start time","End time","Start datetime","End datetime"])
        #print('made candle')
        #tickDict[tick].addCandle(candle)
        #print('added candle')
        if tickDict[tick].callBoughtPrice!=0 and tickDict[tick].soldCallPrice==0 and datetime.datetime.utcnow().time()<datetime.time(19,55) and datetime.datetime.utcnow().time() >datetime.time(13,29):
          tickDict[tick].minutecount+=1
          tickDict[tick].updateStopLoss(cp='call',logs=logs,verbose=True)
          now = datetime.datetime.now()
          if now.hour>18 and now.minute>57:
              tickDict[tick].takeProfit = 0
          tickDict[tick].soldCallPrice = tickDict[tick].check_sell_call(accountslist = accounts_list,logs=logs, verbose=True)
          print(f'{tick} check sell call: {tickDict[tick].soldCallPrice}')
        if tickDict[tick].putBoughtPrice!=0 and tickDict[tick].soldPutPrice==0 and datetime.datetime.utcnow().time()<datetime.time(19,55) and datetime.datetime.utcnow().time() >datetime.time(13,29):
          tickDict[tick].minutecount+=1
          tickDict[tick].updateStopLoss(cp='put',logs=logs,verbose=True)
          now = datetime.datetime.now()
          if now.hour>18 and now.minute>57:
              tickDict[tick].takeProfit = 0
          tickDict[tick].soldPutPrice = tickDict[tick].check_sell_put(accountslist = accounts_list,logs=logs, verbose=True)
          print(f'{tick} check sell put: {tickDict[tick].soldPutPrice}')
        else:
            logs.append(datetime.datetime.now().timestamp(),f"No change to {tick}")
        print(tickDict[tick].ticker+" selling: "+str(time.time()-start_time))
        selling_time = time.time()
        if datetime.datetime.utcnow().time() >datetime.time(13,29):
            if tickDict[tick].updateChecklist(ropen=open,rhigh=high,rlow=low,rclose=close,logs=logs):
                print('entered loop')
                logs.append(datetime.datetime.now().timestamp(),[tick,tickDict[tick].checklist])
                if tickDict[tick].get_oacot(logs=logs) and tickDict[tick].trend=='up' and tickDict[tick].get_oib(logs=logs) and tickDict[tick].callBoughtPrice==0 and tickDict[tick].soldCallPrice==0 and datetime.datetime.utcnow().time() >datetime.time(13,29):
                    print('buy call')
                    tickDict[tick].optionName = tickDict[tick].buy_call(logs=logs,accountslist = accounts_list,verbose = True)
                    options_dict['O:'+tickDict[tick].optionName] = [datetime.datetime.utcnow()-datetime.timedelta(hours=4),tickDict[tick].callBoughtPrice]
                    print(tickDict[tick].optionName)
                if tickDict[tick].get_oacub(logs=logs) and tickDict[tick].trend=='down' and tickDict[tick].get_oib(logs=logs) and tickDict[tick].putBoughtPrice==0 and tickDict[tick].soldPutPrice==0 and datetime.datetime.utcnow().time() >datetime.time(13,29):
                    print('buy put')
                    tickDict[tick].optionName = tickDict[tick].buy_put(logs=logs,accountslist = accounts_list,verbose = True)
                    options_dict['O:'+tickDict[tick].optionName] = [datetime.datetime.utcnow()-datetime.timedelta(hours=4),tickDict[tick].putBoughtPrice]
                    print(tickDict[tick].optionName)
        print(tickDict[tick].ticker+" buying: "+str(time.time()-selling_time))
        buying_time = time.time()
        if datetime.datetime.now().timestamp()-last_save>=300000:
            datetime_format = logs.datetime_format()
            log_file = datetime.datetime.now().strftime(f'%m-%d-%Y')+'_logs.csv'
            important_file = datetime.datetime.now().strftime(f'%m-%d-%Y')+'_important_logs.csv'
            important_logs = datetime_format.where(datetime_format.Message.str.contains("for")).dropna()
            datetime_format.to_csv(f'Logs/{log_file}')
            important_logs.to_csv(f'Logs/{important_file}')
            last_save = 0
        print(tickDict[tick].ticker+" logging: "+str(time.time()-buying_time))
        logging_time = time.time()
        if datetime.datetime.utcnow().time() >datetime.time(19,55):
            for x in tickDict.values():
                if x.callBoughtPrice > 0 and x.soldCallPrice == 0:
                    client = tda.auth.client_from_token_file(f'tokens/kiran.json',config.TD_CLIENT_ID)
                    price = client.get_quote(x.td_option_name).json()[x.td_option_name]['mark']
                    profit = price-x.callBoughtPrice
                    percent_profit = profit/x.callBoughtPrice
                    logs.append(datetime.datetime.now().timestamp(),f'Sell {x.optionName} at {datetime.datetime.now()} for {price}. Profit: {profit} | {percent_profit}%')
                    x.soldCallPrice = price
                    for y in accounts_list:
                        y.sell_option(x.td_option_name,verbose=True, price=price, boughtprice=x.callBoughtPrice)
                if x.putBoughtPrice > 0 and x.soldPutPrice == 0:
                    client = tda.auth.client_from_token_file(f'tokens/kiran.json',config.TD_CLIENT_ID)
                    price = client.get_quote(x.td_option_name).json()[x.td_option_name]['mark']
                    profit = price-x.putBoughtPrice
                    percent_profit = profit/x.putBoughtPrice
                    logs.append(datetime.datetime.now().timestamp(),f'Sell {x.optionName} at {datetime.datetime.now()} for {price}. Profit: {profit} | {percent_profit}%')
                    x.soldPutPrice = price
                    for y in accounts_list:
                        y.sell_option(x.td_option_name,verbose=True,price=price, boughtprice=x.putBoughtPrice)
        print(tickDict[tick].ticker+" ending: "+str(time.time()-logging_time))
    except Exception as e:
        a,b,c = sys.exc_info()
        print(e,message,a,b)


def on_close(ws, close_status_code, close_msg):
    print("on_close args:")
    if close_status_code or close_msg:
        print("close status code: " + str(close_status_code))
        print("close message: " + str(close_msg))

def on_error(ws, error):
    print(error)

ws = websocket.WebSocketApp("wss://stream.data.alpaca.markets/v2/iex", on_open=on_open, on_message=on_message, on_close=on_close,on_error=on_error)

# Get the TickList and Limits

Take a list of tickers and turn it into a dictionary of tickers and stock items after filtering for in-out bars.


Live run starts here

In [32]:
logs = log()
removeList = []
options_dict = {}

In [33]:
tickDict = {}
ticklist = tickers.ticklist
ticklist = set(ticklist)
for x in ticklist:
    y = stock(x,allocation=10,TPpct=.20,SLpct=.20, sldict = {30:.15, 60:.1})
    print(x)
    y.getPastTwoDaysCandles()
    if x not in removeList:
        if y.checkPastTwoDays(logs=logs):
            print(f'{x} is in out')
            tickDict[x] = y
            tickDict[x].reload_checklist()
            if datetime.datetime.utcnow().time() >datetime.time(13,30):
                tickDict[x].checkOpen(logs)
            print(tickDict[x].checklist)
        else:
            print(f"{x} is not in out")

streamlist = [x.stream for x in list(tickDict.values())]

GME
GME is not in out
BA
BA is not in out
GM
GM is not in out
RBLX
RBLX is not in out
PFE
PFE is not in out
AAPL
AAPL is not in out
EBAY
EBAY is not in out
SNAP
SNAP is not in out
SPY
SPY is not in out
F
2022-09-01 13:47:55.119121 F set ioc : True
F is in out
{'InOutCandles': True, 'Opened in between': False, 'Touches Top Line': False, 'Touches Bottom Line': False, 'Open and close over top': False, 'Open and close under bottom': False}
CMCSA
2022-09-01 13:47:55.366121 CMCSA set ioc : True
CMCSA is in out
{'InOutCandles': True, 'Opened in between': False, 'Touches Top Line': False, 'Touches Bottom Line': False, 'Open and close over top': False, 'Open and close under bottom': False}
PYPL
PYPL is not in out
VZ
VZ is not in out
PG
PG is not in out
UBER
UBER is not in out
AAL
AAL is not in out
MSFT
MSFT is not in out
CAT
CAT is not in out
TWTR
TWTR is not in out
DIS
2022-09-01 13:47:55.872151 DIS set ioc : True
DIS is in out
{'InOutCandles': True, 'Opened in between': False, 'Touches Top Li

In [34]:
watchlist.make_watchlist(client=client,tickers=list(tickDict.keys()))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Save logs

In [35]:
datetime_format = logs.datetime_format()
log_file = datetime.datetime.now().strftime(f'%m-%d-%Y')+'_logs.csv'
important_file = datetime.datetime.now().strftime(f'%m-%d-%Y')+'_important_logs.csv'
important_logs = datetime_format.where(datetime_format.Message.str.contains("for")).dropna()
datetime_format.to_csv(f'Logs/{log_file}')
important_logs.to_csv(f'Logs/{important_file}')

Show the limits

In [36]:
dtf = logs.datetime_format()
dtf.where(dtf.Message.str.contains("Limit")).dropna()

,Time,Message
0,2022-09-01 13:47:55.118246,Limits for F: 15.52 | 15.24
3,2022-09-01 13:47:55.364690,Limits for CMCSA: 36.47 | 36.03
6,2022-09-01 13:47:55.870676,Limits for DIS: 113.75 | 111.85
9,2022-09-01 13:47:56.029021,Limits for SOFI: 6.08 | 5.87
12,2022-09-01 13:47:56.215750,Limits for S: 30.0 | 27.16
15,2022-09-01 13:47:56.369167,Limits for MRNA: 139.88 | 131.71
18,2022-09-01 13:47:56.593807,Limits for NIO: 20.38 | 19.28
21,2022-09-01 13:47:56.788908,Limits for LCID: 15.5185 | 14.96
24,2022-09-01 13:47:56.955123,Limits for ABNB: 115.58 | 112.4
27,2022-09-01 13:47:57.156346,Limits for MU: 57.155 | 55.81


# Run Websocket

Create the account list and get the allocations for each account

In [37]:
accounts_list = [paper_trade_account('Test',7422.5,.8),trade_account('kiran',accounts.accounts['kiran'],1)]

In [38]:
for x in accounts_list:
      x.get_account_value()
      x.get_allocation(len(tickDict))
      print(x.allocation)

494
26


In [39]:
ws.close()

In [ ]:
ws.run_forever()

opened
received
'S' [{"T":"success","msg":"connected"}] <class 'KeyError'> 'S'
received
'S' [{"T":"success","msg":"authenticated"}] <class 'KeyError'> 'S'
received
'S' [{"T":"subscription","trades":[],"quotes":[],"bars":["SOFI","S","LCID","PTON","SBUX","CMCSA","F","NIO","ABNB","MU","DIS","MRNA"],"updatedBars":[],"dailyBars":[],"statuses":[],"lulds":[],"corrections":[],"cancelErrors":[]}] <class 'KeyError'> 'S'
received
SBUX
SBUX selling: 0.003907918930053711
2022-09-01 13:49:00.047022 SBUX set tbl : True
entered loop
SBUX buying: 0.01123356819152832
SBUX logging: 0.006700038909912109
SBUX ending: 1.1444091796875e-05
received
MU
MU selling: 0.0028846263885498047
2022-09-01 13:49:00.065831 MU set oacub : True
2022-09-01 13:49:00.068382 MU set tbl : True
entered loop
MU buying: 0.012569189071655273
MU logging: 0.010634899139404297
MU ending: 9.298324584960938e-06
received
ABNB
ABNB selling: 0.0027604103088378906
2022-09-01 13:49:00.089599 ABNB set oib : True
entered loop
ABNB buying: 0.01

In [31]:
for x in accounts_list:
      print(x.get_account_value())

7422.5
312.83


In [429]:
options_data.get_options_high_low(options_dict,'options_data.csv',start_df = pd.read_csv('options_data.csv',index_col=0))
df = pd.read_csv('options_data.csv')
df.describe()

,bought_price,high,low,percent_high,percent_low
count,53.000000,53.000000,53.000000,53.000000,53.000000
mean,1.053019,1.441132,0.588302,0.351831,0.483302
std,1.114652,1.765102,0.694447,0.424192,0.180990
min,0.020000,0.030000,0.010000,-0.700000,0.128492
25%,0.240000,0.360000,0.110000,0.087719,0.342105
50%,0.610000,0.860000,0.280000,0.212766,0.500000
75%,1.700000,1.860000,0.880000,0.500000,0.611111
max,5.350000,10.280000,2.920000,1.619048,0.864407


In [428]:
df

,Unnamed: 0,bought_price,high,low,percent_high,percent_low
0,O:DIS220708C00099000 (07-07-2022),0.67,1.00,0.20,0.492537,0.701493
1,O:LOW220708C00185000 (07-07-2022),0.46,0.76,0.19,0.652174,0.586957
2,O:PFE220708P00050500 (07-07-2022),0.50,0.70,0.20,0.400000,0.600000
3,O:ABNB220708C00093000 (07-07-2022),2.07,4.35,0.75,1.101449,0.637681
4,O:BAC220708P00030000 (07-07-2022),0.32,0.37,0.13,0.156250,0.593750
5,O:GM220708P00030500 (07-07-2022),0.46,0.86,0.17,0.869565,0.630435
6,O:LYFT220708C00014500 (07-07-2022),0.17,0.22,0.06,0.294118,0.647059
7,O:AFRM220708C00020000 (07-07-2022),0.59,0.89,0.08,0.508475,0.864407
8,O:GME220708P00118000 (07-07-2022),3.53,4.80,2.81,0.359773,0.203966
9,O:AFRM220708C00021000 (07-07-2022),0.66,0.74,0.33,0.121212,0.500000


In [324]:
from polygon import RESTClient

pclient = RESTClient(config.POLYGON_KEY)

# View logs

Check account values

In [21]:
datetime_format = logs.datetime_format()
log_file = datetime.datetime.now().strftime(f'%m-%d-%Y')+'_logs.csv'
datetime_format.to_csv(f'Xavier/Logs/{log_file}')
datetime_format

See which tickers have made plays

In [67]:
for x in tickDict.values():
  if x.soldCallPrice==0 and x.soldPutPrice==0:
    print(x.ticker)
  else:
    print(f'No {x.ticker}')

No MS
No KO
No MU
WFC
No JPM
BAC
No NEGG
BA
No SOFI
No AAL
ABNB
